In [29]:
from __future__ import print_function, division

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, utils
import time
import os
import copy
import torch.nn as nn
import torch.nn.functional as F
import datetime

net_param_set = [{'conv':[(),
                          (3, 16, 5, 1, 1), 
                          (16, 32, 3, 1, 1),
                          (32, 32, 3, 1, 0),
                          (32, 16, 3, 1, 1)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             },
             {'conv':[(), 
                      (3, 32, 5, 1, 1), 
                      (32, 32, 3, 1, 0),
                      (32, 16, 3, 1, 1)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             },
             {'conv':[(), 
                      (3, 32, 5, 1, 1), 
                      (32, 16, 3, 1, 0)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             }]
             
epoch_max = 30

optimizer_param_set = [(1e-4, 'expo'),
                       (5e-4, 'expo'),
                       (1e-5, 'step'), 
                       (1e-4, 'step'), 
                       (5e-4, 'step'),
                       (1e-3, 'step')] # learning rate, decay strategy 

In [30]:
def test(model):
    
    # overall training correct rate
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(dataloaders['train'], 0):
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Training accuracy: %d %%' % (100 * correct / total))
    
    # overall testing correct rate
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(dataloaders['test'], 0):
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Testing accuracy: %d %%' % (100 * correct / total))
    
    # count testing predictions for each class
    correct_pred = {classname: 0 for classname in class_names}
    total_pred = {classname: 0 for classname in class_names}
    with torch.no_grad():
        for i, data in enumerate(dataloaders['test'], 0):
            images, labels = data
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[class_names[label]] += 1
                total_pred[class_names[label]] += 1

    # print accuracy for each class
    print("Testing accuracy (each class): ")
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print("{:1s}: {:.1f}%;  ".format(classname, accuracy), end=' ')
        if classname == "5":
            print()
    print()
    
    return


def train_test(model, criterion, optimizer, scheduler, num_epochs=25):
    
    for epoch in range(num_epochs):  

        running_loss = 0.0
        loss_record=[]
        for i, data in enumerate(dataloaders['train'], 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                loss_record.append(round(running_loss / 2000, 8))
                # print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
                
        print(datetime.datetime.now(), ' Epoch', (epoch + 1), ': Average Loss', loss_record)

    print('Finished Training')
    
    test(model)
    
    return None

# Data transformer
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Dataset initialization
data_dir = 'data' 
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']} # Read train and test sets, respectively.

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0) for x in ['train', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Set device to "cpu" if you have no gpu


In [31]:
class Net(nn.Module):
    """
    Input - 1x32x32
    Output - 10
    CONV1->CONV2->POOL1->CONV3->CONV4->POOL2->FC1->FC2->FC3
    """
    def __init__(self, params):
        super(Net, self).__init__()
        
        #Initialize layers
        self.params = params
        self.n_layers = len(params['conv'])+len(params['pool'])+len(params['fc'])-3
        self.printed = False
        
        if self.n_layers == 9:
            self.conv1 = nn.Conv2d(*self.params['conv'][1])
            self.conv2 = nn.Conv2d(*self.params['conv'][2])
            self.conv3 = nn.Conv2d(*self.params['conv'][3])
            self.conv4 = nn.Conv2d(*self.params['conv'][4])
            
            self.pool1 = nn.MaxPool2d(*self.params['pool'][1])
            self.pool2 = nn.MaxPool2d(*self.params['pool'][2])
            
            self.fc1 = nn.Linear(*self.params['fc'][1])
            self.fc2 = nn.Linear(*self.params['fc'][2])
            self.fc3 = nn.Linear(*self.params['fc'][3])
            
        elif self.n_layers == 8:
            self.conv1 = nn.Conv2d(*self.params['conv'][1])
            self.conv2 = nn.Conv2d(*self.params['conv'][2])
            self.conv3 = nn.Conv2d(*self.params['conv'][3])
            
            self.pool1 = nn.MaxPool2d(*self.params['pool'][1])
            self.pool2 = nn.MaxPool2d(*self.params['pool'][2])
            
            self.fc1 = nn.Linear(*self.params['fc'][1])
            self.fc2 = nn.Linear(*self.params['fc'][2])
            self.fc3 = nn.Linear(*self.params['fc'][3])
        
        elif self.n_layers == 7:
            self.conv1 = nn.Conv2d(*self.params['conv'][1])
            self.conv2 = nn.Conv2d(*self.params['conv'][2])
            
            self.pool1 = nn.MaxPool2d(*self.params['pool'][1])
            self.pool2 = nn.MaxPool2d(*self.params['pool'][2])
            
            self.fc1 = nn.Linear(*self.params['fc'][1])
            self.fc2 = nn.Linear(*self.params['fc'][2])
            self.fc3 = nn.Linear(*self.params['fc'][3])
            

    def forward(self, img):
        # Implement forward pass
        x = img
        
        if self.n_layers == 9:
            x = F.relu(self.conv1(x))
            if not self.printed: 
                print("CONV1", x.size(), end=" || ")
            x = F.relu(self.conv2(x))
            if not self.printed: 
                print("CONV2", x.size(), end=" || ")
            x = self.pool1(x)
            if not self.printed: 
                print("POOL1", x.size())
        
            x = F.relu(self.conv3(x))
            if not self.printed: 
                print("CONV3", x.size(), end=" || ")
            x = F.relu(self.conv4(x))
            if not self.printed: 
                print("CONV4", x.size(), end=" || ")
            x = self.pool2(x)
            if not self.printed: 
                print("POOL2", x.size())
        
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            if not self.printed: 
                print("FC1", x.size(), end=" || ")
            x = F.relu(self.fc2(x))
            if not self.printed: 
                print("FC2", x.size(), end=" || ")
            x = self.fc3(x)
            if not self.printed: 
                print("FC3", x.size())
                
        elif self.n_layers == 8:
            x = F.relu(self.conv1(x))
            if not self.printed: 
                print("CONV1", x.size(), end=" || ")
            x = self.pool1(x)
            if not self.printed: 
                print("POOL1", x.size())
        
            x = F.relu(self.conv2(x))
            if not self.printed: 
                print("CONV2", x.size(), end=" || ")
            x = F.relu(self.conv3(x))
            if not self.printed: 
                print("CONV3", x.size(), end=" || ")
            x = self.pool2(x)
            if not self.printed: 
                print("POOL2", x.size())
        
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            if not self.printed: 
                print("FC1", x.size(), end=" || ")
            x = F.relu(self.fc2(x))
            if not self.printed: 
                print("FC2", x.size(), end=" || ")
            x = self.fc3(x)
            if not self.printed: 
                print("FC3", x.size())
        
        elif self.n_layers == 7:
            x = F.relu(self.conv1(x))
            if not self.printed: 
                print("CONV1", x.size(), end=" || ")
            x = self.pool1(x)
            if not self.printed: 
                print("POOL1", x.size())
        
            x = F.relu(self.conv2(x))
            if not self.printed: 
                print("CONV2", x.size(), end=" || ")
            x = self.pool2(x)
            if not self.printed: 
                print("POOL2", x.size())
        
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            if not self.printed: 
                print("FC1", x.size(), end=" || ")
            x = F.relu(self.fc2(x))
            if not self.printed: 
                print("FC2", x.size(), end=" || ")
            x = self.fc3(x)
            if not self.printed: 
                print("FC3", x.size())
        
        self.printed = True

        return x


In [32]:
paths = []
for j in range(len(optimizer_param_set)):
    for i in range(len(net_param_set)):
        model_ft = Net(net_param_set[i])
        model_ft = model_ft.to(device)
        criterion = nn.CrossEntropyLoss()
        learning_rate, decay_strategy = optimizer_param_set[j]
        optimizer_ft = optim.Adam(model_ft.parameters(), lr=learning_rate)
        if decay_strategy=='expo':
            lr_scheduler_ft = lr_scheduler.ExponentialLR(optimizer_ft, gamma=0.1)
        elif decay_strategy=='step':
            lr_scheduler_ft = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)
        else:
            print("ERROR 2")
        
        print(i, ",", j)
        for n in range((epoch_max // 5)):
            print(datetime.datetime.now())
            epo = 5*n+5
            print("epoch range: ", epo-4, " to ", epo)
            train_test(model_ft, criterion, optimizer_ft, lr_scheduler_ft, num_epochs=5)
        
        PATH = "./cnn"+str(i)+"_"+str(j)+"_.pth"
        paths.append(PATH)
        torch.save({'epoch': epoch_max,
                    'model_state_dict': model_ft.state_dict(),
                    'optimizer_state_dict': optimizer_ft.state_dict()
                   }, PATH)


0 , 0
2021-05-08 19:33:58.092180
epoch range:  1  to  5
CONV1 torch.Size([4, 16, 30, 30]) || CONV2 torch.Size([4, 32, 30, 30]) || POOL1 torch.Size([4, 32, 15, 15])
CONV3 torch.Size([4, 32, 13, 13]) || CONV4 torch.Size([4, 16, 13, 13]) || POOL2 torch.Size([4, 16, 6, 6])
FC1 torch.Size([4, 120]) || FC2 torch.Size([4, 90]) || FC3 torch.Size([4, 10])
2021-05-08 19:34:49.508695  Epoch 1 : Average Loss [2.10735773, 1.43312485, 1.25826324]
2021-05-08 19:35:40.025076  Epoch 2 : Average Loss [1.04885132, 0.96049483, 0.86992286]
2021-05-08 19:36:25.787474  Epoch 3 : Average Loss [0.7811505, 0.73868888, 0.7229372]
2021-05-08 19:37:13.810075  Epoch 4 : Average Loss [0.64957764, 0.61825972, 0.62710748]
2021-05-08 19:38:06.031005  Epoch 5 : Average Loss [0.56425708, 0.54605706, 0.56628655]
Finished Training
Training accuracy: 83 %
Testing accuracy: 81 %
Testing accuracy (each class): 
0: 87.8%;   1: 81.8%;   2: 77.0%;   3: 73.4%;   4: 88.2%;   5: 78.8%;   
6: 75.6%;   7: 86.6%;   8: 86.2%;   9: 79.0

Testing accuracy (each class): 
0: 90.0%;   1: 86.6%;   2: 86.8%;   3: 75.0%;   4: 94.4%;   5: 84.8%;   
6: 85.2%;   7: 93.2%;   8: 85.2%;   9: 86.8%;   
2021-05-08 20:21:31.247285
epoch range:  26  to  30
2021-05-08 20:22:12.890471  Epoch 1 : Average Loss [0.17023362, 0.17812215, 0.18498417]
2021-05-08 20:22:52.583791  Epoch 2 : Average Loss [0.16618216, 0.16459052, 0.17739516]
2021-05-08 20:23:34.505301  Epoch 3 : Average Loss [0.14596712, 0.1722998, 0.16109413]
2021-05-08 20:24:19.642439  Epoch 4 : Average Loss [0.14191268, 0.14733252, 0.17162771]
2021-05-08 20:25:03.951659  Epoch 5 : Average Loss [0.13934712, 0.1476462, 0.15772364]
Finished Training
Training accuracy: 95 %
Testing accuracy: 85 %
Testing accuracy (each class): 
0: 84.6%;   1: 86.8%;   2: 84.2%;   3: 80.8%;   4: 92.4%;   5: 84.4%;   
6: 83.4%;   7: 89.0%;   8: 83.8%;   9: 88.8%;   
2 , 0
2021-05-08 20:25:28.811185
epoch range:  1  to  5
CONV1 torch.Size([4, 32, 30, 30]) || POOL1 torch.Size([4, 32, 15, 15])
CONV2 torc

Training accuracy: 92 %
Testing accuracy: 87 %
Testing accuracy (each class): 
0: 88.8%;   1: 92.2%;   2: 87.0%;   3: 83.6%;   4: 90.6%;   5: 84.4%;   
6: 87.8%;   7: 88.8%;   8: 85.4%;   9: 88.6%;   
2021-05-08 21:06:04.138228
epoch range:  21  to  25
2021-05-08 21:06:57.775864  Epoch 1 : Average Loss [0.24709209, 0.2280115, 0.26215534]
2021-05-08 21:07:48.927849  Epoch 2 : Average Loss [0.22189146, 0.24629876, 0.24532284]
2021-05-08 21:08:44.674598  Epoch 3 : Average Loss [0.22386449, 0.24721279, 0.2478231]
2021-05-08 21:09:41.245265  Epoch 4 : Average Loss [0.2236789, 0.21587964, 0.26378473]
2021-05-08 21:10:31.755430  Epoch 5 : Average Loss [0.21565433, 0.22195338, 0.25799202]
Finished Training
Training accuracy: 93 %
Testing accuracy: 86 %
Testing accuracy (each class): 
0: 86.2%;   1: 90.2%;   2: 86.0%;   3: 86.6%;   4: 89.6%;   5: 88.0%;   
6: 79.8%;   7: 85.8%;   8: 84.4%;   9: 89.4%;   
2021-05-08 21:10:59.960157
epoch range:  26  to  30
2021-05-08 21:11:56.071130  Epoch 1 : A

Training accuracy: 91 %
Testing accuracy: 84 %
Testing accuracy (each class): 
0: 90.8%;   1: 86.8%;   2: 86.8%;   3: 66.6%;   4: 90.2%;   5: 85.4%;   
6: 83.0%;   7: 90.6%;   8: 80.8%;   9: 80.8%;   
2021-05-08 21:52:46.276681
epoch range:  16  to  20
2021-05-08 21:53:26.774003  Epoch 1 : Average Loss [0.26073387, 0.2972288, 0.30749867]
2021-05-08 21:54:03.912378  Epoch 2 : Average Loss [0.28722132, 0.27625257, 0.27838565]
2021-05-08 21:54:42.995715  Epoch 3 : Average Loss [0.25315745, 0.28520962, 0.27821338]
2021-05-08 21:55:24.606623  Epoch 4 : Average Loss [0.2526744, 0.27871163, 0.26441]
2021-05-08 21:56:09.568950  Epoch 5 : Average Loss [0.23923765, 0.24330983, 0.27404666]
Finished Training
Training accuracy: 92 %
Testing accuracy: 84 %
Testing accuracy (each class): 
0: 89.2%;   1: 86.8%;   2: 87.2%;   3: 76.4%;   4: 85.0%;   5: 86.0%;   
6: 84.0%;   7: 89.4%;   8: 83.0%;   9: 81.4%;   
2021-05-08 21:56:33.616454
epoch range:  21  to  25
2021-05-08 21:57:12.190360  Epoch 1 : Ave

2021-05-08 22:39:12.293956  Epoch 5 : Average Loss [1.24839888, 1.2157367, 1.22184783]
Finished Training
Training accuracy: 60 %
Testing accuracy: 58 %
Testing accuracy (each class): 
0: 69.2%;   1: 70.0%;   2: 52.2%;   3: 38.2%;   4: 67.0%;   5: 53.6%;   
6: 53.0%;   7: 79.2%;   8: 58.6%;   9: 48.2%;   
2021-05-08 22:39:38.553847
epoch range:  11  to  15
2021-05-08 22:40:22.467041  Epoch 1 : Average Loss [1.20714333, 1.19728428, 1.17858209]
2021-05-08 22:41:06.752120  Epoch 2 : Average Loss [1.18401816, 1.16343505, 1.15973496]
2021-05-08 22:41:55.450018  Epoch 3 : Average Loss [1.1714133, 1.12363357, 1.1414791]
2021-05-08 22:42:48.621666  Epoch 4 : Average Loss [1.11801023, 1.14016037, 1.10491116]
2021-05-08 22:43:33.803769  Epoch 5 : Average Loss [1.10213437, 1.09488659, 1.09089617]
Finished Training
Training accuracy: 64 %
Testing accuracy: 63 %
Testing accuracy (each class): 
0: 69.8%;   1: 77.0%;   2: 66.8%;   3: 32.6%;   4: 77.4%;   5: 58.6%;   
6: 50.0%;   7: 80.0%;   8: 67.0%; 

2021-05-08 23:19:29.832078  Epoch 4 : Average Loss [0.73406446, 0.71743294, 0.66955047]
2021-05-08 23:20:21.632215  Epoch 5 : Average Loss [0.59486343, 0.61315687, 0.58070013]
Finished Training
Training accuracy: 83 %
Testing accuracy: 80 %
Testing accuracy (each class): 
0: 88.4%;   1: 81.0%;   2: 79.2%;   3: 71.6%;   4: 87.8%;   5: 73.4%;   
6: 71.0%;   7: 90.6%;   8: 78.2%;   9: 82.4%;   
2021-05-08 23:20:49.409824
epoch range:  6  to  10
2021-05-08 23:21:35.117829  Epoch 1 : Average Loss [0.54430703, 0.52305112, 0.527829]
2021-05-08 23:22:24.310202  Epoch 2 : Average Loss [0.49992622, 0.48357654, 0.49075223]
2021-05-08 23:23:17.064193  Epoch 3 : Average Loss [0.45509478, 0.44518489, 0.4575458]
2021-05-08 23:24:05.603710  Epoch 4 : Average Loss [0.40732438, 0.42939543, 0.42000879]
2021-05-08 23:24:53.401659  Epoch 5 : Average Loss [0.38370412, 0.40114585, 0.39701254]
Finished Training
Training accuracy: 88 %
Testing accuracy: 85 %
Testing accuracy (each class): 
0: 89.2%;   1: 88.2%

Training accuracy: 95 %
Testing accuracy: 85 %
Testing accuracy (each class): 
0: 88.8%;   1: 86.4%;   2: 91.8%;   3: 77.4%;   4: 87.8%;   5: 82.8%;   
6: 81.4%;   7: 90.0%;   8: 86.0%;   9: 85.0%;   
2 , 3
2021-05-09 00:11:41.890619
epoch range:  1  to  5
CONV1 torch.Size([4, 32, 30, 30]) || POOL1 torch.Size([4, 32, 15, 15])
CONV2 torch.Size([4, 16, 13, 13]) || POOL2 torch.Size([4, 16, 6, 6])
FC1 torch.Size([4, 120]) || FC2 torch.Size([4, 90]) || FC3 torch.Size([4, 10])
2021-05-09 00:12:19.571424  Epoch 1 : Average Loss [2.12657541, 1.46895662, 1.2500565]
2021-05-09 00:12:58.324902  Epoch 2 : Average Loss [0.97621176, 0.9286132, 0.88596126]
2021-05-09 00:13:33.610086  Epoch 3 : Average Loss [0.78687564, 0.77286833, 0.76293396]
2021-05-09 00:14:07.190591  Epoch 4 : Average Loss [0.69874124, 0.68250486, 0.66905456]
2021-05-09 00:14:42.601301  Epoch 5 : Average Loss [0.61225189, 0.6025064, 0.62953426]
Finished Training
Training accuracy: 82 %
Testing accuracy: 80 %
Testing accuracy (each

2021-05-09 01:07:35.624693  Epoch 5 : Average Loss [0.21913517, 0.22163358, 0.24606573]
Finished Training
Training accuracy: 93 %
Testing accuracy: 86 %
Testing accuracy (each class): 
0: 90.4%;   1: 89.4%;   2: 84.2%;   3: 79.8%;   4: 91.0%;   5: 84.2%;   
6: 83.2%;   7: 91.8%;   8: 84.2%;   9: 88.6%;   
2021-05-09 01:08:07.786523
epoch range:  26  to  30
2021-05-09 01:09:07.245071  Epoch 1 : Average Loss [0.21878447, 0.21963469, 0.23275922]
2021-05-09 01:10:06.002027  Epoch 2 : Average Loss [0.21067922, 0.2125717, 0.23443927]
2021-05-09 01:11:04.258239  Epoch 3 : Average Loss [0.21414827, 0.21930565, 0.25557284]
2021-05-09 01:12:35.685701  Epoch 4 : Average Loss [0.18165385, 0.2296636, 0.21933787]
2021-05-09 01:14:56.098607  Epoch 5 : Average Loss [0.20716519, 0.2177972, 0.21225606]
Finished Training
Training accuracy: 93 %
Testing accuracy: 86 %
Testing accuracy (each class): 
0: 89.4%;   1: 86.8%;   2: 86.0%;   3: 78.0%;   4: 91.0%;   5: 83.4%;   
6: 87.4%;   7: 93.0%;   8: 83.0%; 

2021-05-09 02:49:31.611718  Epoch 4 : Average Loss [0.26340812, 0.28380203, 0.29867322]
2021-05-09 02:50:21.423414  Epoch 5 : Average Loss [0.26066848, 0.27467865, 0.27948609]
Finished Training
Training accuracy: 91 %
Testing accuracy: 82 %
Testing accuracy (each class): 
0: 84.6%;   1: 80.4%;   2: 88.0%;   3: 69.8%;   4: 89.0%;   5: 84.8%;   
6: 79.4%;   7: 93.2%;   8: 79.0%;   9: 81.4%;   
2021-05-09 02:50:53.302719
epoch range:  21  to  25
2021-05-09 02:51:44.965049  Epoch 1 : Average Loss [0.2337789, 0.28833726, 0.27435031]
2021-05-09 02:52:29.147676  Epoch 2 : Average Loss [0.24465728, 0.25797386, 0.2953628]
2021-05-09 02:53:25.293960  Epoch 3 : Average Loss [0.23359468, 0.25711054, 0.26447333]
2021-05-09 02:54:13.828604  Epoch 4 : Average Loss [0.24019968, 0.25319723, 0.26077838]
2021-05-09 02:54:56.706655  Epoch 5 : Average Loss [0.22526045, 0.23793618, 0.25408837]
Finished Training
Training accuracy: 92 %
Testing accuracy: 84 %
Testing accuracy (each class): 
0: 88.2%;   1: 85.

KeyboardInterrupt: 

In [33]:
for i in range(len(paths)):
    PATH = paths[i]
    print(PATH)
    
    j = int(PATH[5]) # net parameter
    k = int(PATH[7]) # optimizer parameter
    model = Net(net_param_set[j])
    learning_rate, decay_strategy = optimizer_param_set[k]
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    
    model.eval()
    test(model)
    print()


./cnn0_0_.pth
CONV1 torch.Size([4, 16, 30, 30]) || CONV2 torch.Size([4, 32, 30, 30]) || POOL1 torch.Size([4, 32, 15, 15])
CONV3 torch.Size([4, 32, 13, 13]) || CONV4 torch.Size([4, 16, 13, 13]) || POOL2 torch.Size([4, 16, 6, 6])
FC1 torch.Size([4, 120]) || FC2 torch.Size([4, 90]) || FC3 torch.Size([4, 10])
Training accuracy: 96 %
Testing accuracy: 87 %
Testing accuracy (each class): 
0: 88.2%;   1: 86.2%;   2: 89.4%;   3: 76.6%;   4: 90.6%;   5: 89.2%;   
6: 86.6%;   7: 92.8%;   8: 86.0%;   9: 91.0%;   

./cnn1_0_.pth
CONV1 torch.Size([4, 32, 30, 30]) || POOL1 torch.Size([4, 32, 15, 15])
CONV2 torch.Size([4, 32, 13, 13]) || CONV3 torch.Size([4, 16, 13, 13]) || POOL2 torch.Size([4, 16, 6, 6])
FC1 torch.Size([4, 120]) || FC2 torch.Size([4, 90]) || FC3 torch.Size([4, 10])
Training accuracy: 95 %


KeyboardInterrupt: 

In [ ]:
net_param_set.append({'conv':[(),
                          (3, 32, 5, 1, 0), 
                          (32, 16, 3, 1, 0)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             })